# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [67]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [68]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [69]:
print(orders.shape)
orders.dtypes

(397924, 14)


Unnamed: 0        int64
InvoiceNo         int64
StockCode        object
year              int64
month             int64
day               int64
hour              int64
Description      object
Quantity          int64
InvoiceDate      object
UnitPrice       float64
CustomerID        int64
Country          object
amount_spent    float64
dtype: object

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [84]:
# your code here
#Sub Problem 1
agg_amt = pd.DataFrame(orders.groupby(['CustomerID', 'Country']).agg({'amount_spent': 'sum'}).sort_values(['amount_spent'], ascending = False).reset_index())
agg_amt

,CustomerID,Country,amount_spent
0,14646,Netherlands,280206.02
1,18102,United Kingdom,259657.30
2,17450,United Kingdom,194550.79
3,16446,United Kingdom,168472.50
4,14911,EIRE,143825.06
...,...,...,...
4342,17956,United Kingdom,12.75
4343,16454,United Kingdom,6.90
4344,14792,United Kingdom,6.20
4345,16738,United Kingdom,3.75


In [59]:
#Sub Problem 2 + Sub Problem 3
# VIP Customers: aggregated expenses > 95th percentile
# Preferred Customers: aggregated expenses > 75th and =< 95th percentile


In [85]:
q75 = agg_amt['amount_spent'].quantile(0.75)
q95 = agg_amt['amount_spent'].quantile(0.95)
print(q75)
print(q95)

1661.195
5774.277999999996


In [86]:
def customer_category(row):
    if (row['amount_spent'] > q75) & (row['amount_spent'] <= q95):
        return 'Preferred Customer'
    elif row['amount_spent'] > q95:
        return 'VIP Customer'
    else:
        return 'Other'

In [87]:
agg_amt['Customer category'] = agg_amt.apply(customer_category, axis = 1)
agg_amt.sample(5)

,CustomerID,Country,amount_spent,Customer category
2233,15651,United Kingdom,646.30,Other
341,14277,France,4216.20,Preferred Customer
1551,15763,United Kingdom,1086.92,Other
2873,18067,United Kingdom,396.33,Other
307,15093,United Kingdom,4597.02,Preferred Customer


In [88]:
agg_amt['Customer category'].value_counts()

Other                 3260
Preferred Customer     869
VIP Customer           218
Name: Customer category, dtype: int64

In [89]:
agg_amt.drop(agg_amt.loc[agg_amt['Customer category'] == 'Other'].index, inplace = True)
agg_amt['Customer category'].value_counts()

Preferred Customer    869
VIP Customer          218
Name: Customer category, dtype: int64

In [90]:
agg_amt.shape

(1087, 4)

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [106]:
# your code here --> United Kingdom has the most VIP Customers

agg_amt.groupby(['Country', 'Customer category'])['CustomerID'].agg('count').reset_index().sort_values(['CustomerID'], ascending = False)

,Country,Customer category,CustomerID
40,United Kingdom,Preferred Customer,756
41,United Kingdom,VIP Customer,178
17,Germany,Preferred Customer,28
15,France,Preferred Customer,20
3,Belgium,Preferred Customer,11
18,Germany,VIP Customer,11
16,France,VIP Customer,9
38,Switzerland,Preferred Customer,6
28,Norway,Preferred Customer,6
34,Spain,Preferred Customer,5


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [94]:
# your code here --> United Kingdom has the most VIP+Preferred Customers combined
agg_amt['Country'].value_counts()

United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Portugal             7
Norway               7
Spain                7
Finland              5
Italy                5
Japan                4
Channel Islands      4
Australia            4
Denmark              3
Cyprus               3
EIRE                 3
Sweden               2
Israel               2
Malta                1
Canada               1
Greece               1
Austria              1
Netherlands          1
Poland               1
Iceland              1
Singapore            1
Lebanon              1
Name: Country, dtype: int64